In [1]:
from transformers import (
    BertModel, 
    AutoConfig, 
    AutoTokenizer, 
    Trainer,
    TrainingArguments
    )
import torch.nn as nn
import datasets
import csv


In [2]:
import torch
import numpy as np

torch.cuda.is_available()

True

In [3]:
import csv

In [4]:
## Load dataset
feats_fp = open("BERT_X.csv", "r")
labels_fp = open("BERT_y.csv", "r")
feats = csv.reader(feats_fp)
labels = csv.reader(labels_fp)

# skip header
next(feats)
next(labels)

unscaled_data = {'text': [], 'label': []}
nn_data = []
for row in feats:
    unscaled_data['text'].append(row[0].strip().replace("\n", " "))
    nn_feats = [float(col.strip().replace("\n", "")) for col in row[1:]]
    
    
    nn_data.append(nn_feats)
for i,row in enumerate(labels):
    unscaled_data['label'].append(float(row[0].strip().replace("\n", "")))
nn_data = np.array(nn_data).astype(np.float32)
nn_data[0]
    

array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 1.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_labels = scaler.fit_transform(np.array(unscaled_data['label']).reshape(-1,1)).flatten()
scaled_data = unscaled_data
scaled_data['label'] = scaled_labels

employee_count_scaler = StandardScaler()
scaled_employee_count =employee_count_scaler.fit_transform(nn_data[:,-2].reshape(-1,1)).flatten()
nn_data[:,-2] = scaled_employee_count
scaled_labels, nn_data[:,-2]

(array([-0.58786439, -0.94649659,  2.86819603, ..., -1.04441134,
        -0.86218111, -0.69471581]),
 array([-0.30006495, -0.31242564, -0.2994551 , ..., -0.31238207,
        -0.3128286 , -0.31278503], dtype=float32))

In [6]:
assert len(scaled_data['text']) == len(scaled_data['label']) == nn_data.shape[0]
dataset = datasets.Dataset.from_dict(scaled_data)

In [7]:
print(dataset[0]['text'])
scaler.inverse_transform(np.array([dataset[0]['label']]).reshape(-1,1))

Overview  HearingLife is a national hearing care company and part of the Demant Group, a global leader in hearing healthcare built on a heritage of care, health, and innovation since 1904. HearingLife operates more than 600 hearing care centers across 42 states. We follow a scientific, results-oriented approach to hearing healthcare that is provided by highly skilled and caring professionals. Our vision is to help more people hear better through life-changing hearing health delivered by the best personalized care. This Team Member must uphold the HearingLife Core Values:   We create trust  We are team players  We apply a can-do attitude  We create innovative solutions   Responsibilities  You will help more people hear better by providing clinical expertise to diagnose and treat hearing loss while ensuring a positive patient experience. The Hearing Care Provider acts in accordance with required industry and state professional licensing standards and local practice scope and is responsib

array([[63000.]])

In [8]:
# Tokenize the dataset
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
def preprocess(examples):
    return tokenizer(examples['text'], truncation=True, padding=True)

tokenized_dataset = dataset.map(preprocess, batched=True)

  0%|          | 0/27 [00:00<?, ?ba/s]

In [9]:
# training_args = TrainingArguments(
#     output_dir='./results',          
#     num_train_epochs=3,              
#     per_device_train_batch_size=16,  
#     learning_rate=5e-5,               
#     warmup_steps=500,                
#     weight_decay=0.01,              
#     logging_dir='./logs',
# )


In [10]:
tokenized_dataset

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 26990
})

In [11]:
from torch.utils.data import Dataset

class RegressionDataset(Dataset):
    def __init__(self, input_ids, attention_mask, labels, nn_data):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.labels = labels
        self.nn_data = nn_data

    def __getitem__(self, idx):
        item = {
            'input_ids': torch.tensor(self.input_ids[idx]),
            'attention_mask': torch.tensor(self.attention_mask[idx]),
            'labels': torch.tensor(self.labels[idx]),
            'nn_data': torch.tensor(self.nn_data[idx])
        }
        return item

    def __len__(self):
        return len(self.labels)  # Assuming all data entries have labels

    
    

In [12]:
input_ids = tokenized_dataset['input_ids']
attention_mask = tokenized_dataset['attention_mask']
labels = dataset['label']  # Assuming your labels are in the original dataset
nn_data = nn_data

reg_dataset = RegressionDataset(input_ids, attention_mask, labels, nn_data)
dataloader = torch.utils.data.DataLoader(reg_dataset, batch_size=64, shuffle=True)



In [13]:
reg_dataset[0]['nn_data']

tensor([ 0.0000,  0.0000,  0.0000,  0.0000,  1.0000,  1.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  1.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000,  0.00

In [14]:

model_name = "bert-base-uncased"
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
bert_model = BertModel.from_pretrained(model_name)

config.problem_type = 'regression'

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:

# Add linear layer
output_size = 1  


# Combine BERT and the linear layer
class BertWithLinear(nn.Module):
    def __init__(self):
        super(BertWithLinear, self).__init__()
        self.bert = bert_model.to('cuda')
    
        self.nn = nn.Sequential(
            nn.Linear(config.hidden_size + len(nn_data[0]) , 128),
            nn.ReLU(),
            nn.Linear(128, 16),
            nn.ReLU(),
            nn.Linear(16, output_size)
        ).to('cuda')
        

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, inputs_embeds=None, labels=None, nn_data = None):
        output = self.bert(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, position_ids=position_ids, head_mask=head_mask, inputs_embeds=inputs_embeds)
        # Use pooled output for classification/regression
        pooled_output = output.pooler_output
        nn_data = torch.cat((pooled_output, nn_data), 1)
        nn_output = self.nn(nn_data)
        return nn_output

    

model = BertWithLinear().to('cuda')

In [19]:
# freeze/unfreeze BERT pretrained weights
for param in model.bert.embeddings.parameters():
    param.requires_grad = True
model = nn.DataParallel(model)

In [20]:
model.parameters()

<generator object Module.parameters at 0x2afb2b738ca8>

In [21]:
loss_fn = nn.MSELoss()  # Mean Squared Error is common for regression
optimizer = torch.optim.Adam([
    {'params': model.module.bert.parameters(), 'lr': 1e-5}, 
    {'params': model.module.nn.parameters(), 'lr': 2e-5} # our neural net
])
num_epochs = 30
    
    
from torch.utils.data import random_split 

train_size = int(0.8 * len(reg_dataset))  # 80% of the dataset for training
val_size = len(reg_dataset) - train_size 

# Create the train and validation datasets
train_dataset, val_dataset = random_split(reg_dataset, [train_size, val_size])

# Create DataLoaders
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)  # No need to shuffle validation
train_losses = []
val_losses = []
# Modify your training loop (add validation)
for epoch in range(num_epochs):
    print(epoch,"/",num_epochs)
    b=0
    ## Training Phase
    model.train()  # Set model to training mode
    current_epoch_train_losses = []
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to('cuda') 
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')
        data = batch['nn_data'].to('cuda')

        optimizer.zero_grad()
        
        outputs = model(input_ids, attention_mask=attention_mask, nn_data=data)
        
        loss = loss_fn(outputs.squeeze(1), labels) # Ensure outputs are single-dimensional
        current_epoch_train_losses.append(loss.item())
        loss.backward()
        optimizer.step() 
        print(f"batch {b} complete. Loss: ",loss.item())
        b+=1
    train_losses.append(sum(current_epoch_train_losses)/len(current_epoch_train_losses))
    ## Validation Phase
    model.eval()   # Set model to evaluation mode
    val_loss = 0
    with torch.no_grad():  # Disable gradient calculation for validation
        for batch in val_dataloader:
            input_ids = batch['input_ids'].to('cuda') 
            attention_mask = batch['attention_mask'].to('cuda')
            labels = batch['labels'].to('cuda')
            data = batch['nn_data'].to('cuda')
            outputs = model(input_ids, attention_mask=attention_mask,nn_data=data)
            val_loss += loss_fn(outputs.squeeze(1), labels).item()

    val_loss /= len(val_dataloader)
    val_losses.append(val_loss)
    print(f"Epoch {epoch} Validation Loss: {val_loss}")

    # checkpoint model
    if epoch % 5 == 0:
        torch.save(model.state_dict(), f"model_checkpoint_{epoch}.pth")


0 / 30
batch 0 complete. Loss:  0.6949131488800049
batch 1 complete. Loss:  0.7515292167663574
batch 2 complete. Loss:  1.3254096508026123
batch 3 complete. Loss:  0.800460934638977
batch 4 complete. Loss:  0.6648236513137817
batch 5 complete. Loss:  0.7780781984329224
batch 6 complete. Loss:  1.2278398275375366
batch 7 complete. Loss:  1.6387436389923096
batch 8 complete. Loss:  0.7625582814216614
batch 9 complete. Loss:  1.0641542673110962
batch 10 complete. Loss:  1.084274172782898
batch 11 complete. Loss:  0.46518009901046753
batch 12 complete. Loss:  0.9497472047805786
batch 13 complete. Loss:  0.7691526412963867
batch 14 complete. Loss:  1.0183334350585938
batch 15 complete. Loss:  0.6266006231307983
batch 16 complete. Loss:  1.0170462131500244
batch 17 complete. Loss:  0.884953498840332
batch 18 complete. Loss:  1.7094956636428833
batch 19 complete. Loss:  1.29136061668396
batch 20 complete. Loss:  1.1537408828735352
batch 21 complete. Loss:  0.9390724301338196
batch 22 complete

batch 181 complete. Loss:  0.5592057704925537
batch 182 complete. Loss:  0.5389049053192139
batch 183 complete. Loss:  0.37561842799186707
batch 184 complete. Loss:  0.26477688550949097
batch 185 complete. Loss:  0.31196802854537964
batch 186 complete. Loss:  0.5741144418716431
batch 187 complete. Loss:  0.44888174533843994
batch 188 complete. Loss:  0.26336243748664856
batch 189 complete. Loss:  0.7150803804397583
batch 190 complete. Loss:  0.3217500150203705
batch 191 complete. Loss:  0.25530433654785156
batch 192 complete. Loss:  0.5593010187149048
batch 193 complete. Loss:  0.7769856452941895
batch 194 complete. Loss:  0.40545031428337097
batch 195 complete. Loss:  0.41831567883491516
batch 196 complete. Loss:  0.4488863945007324
batch 197 complete. Loss:  0.5676374435424805
batch 198 complete. Loss:  0.48540377616882324
batch 199 complete. Loss:  0.6074104309082031
batch 200 complete. Loss:  0.5662814974784851
batch 201 complete. Loss:  0.5243139266967773
batch 202 complete. Loss:

batch 358 complete. Loss:  0.39933669567108154
batch 359 complete. Loss:  0.180294930934906
batch 360 complete. Loss:  0.24374686181545258
batch 361 complete. Loss:  0.21091996133327484
batch 362 complete. Loss:  0.2175299972295761
batch 363 complete. Loss:  0.15747112035751343
batch 364 complete. Loss:  0.43402451276779175
batch 365 complete. Loss:  0.7754325270652771
batch 366 complete. Loss:  0.27902403473854065
batch 367 complete. Loss:  0.41108596324920654
batch 368 complete. Loss:  0.3851276636123657
batch 369 complete. Loss:  0.4182986617088318
batch 370 complete. Loss:  0.35725080966949463
batch 371 complete. Loss:  0.2468080222606659
batch 372 complete. Loss:  0.41557934880256653
batch 373 complete. Loss:  0.2487785518169403
batch 374 complete. Loss:  0.2940024733543396
batch 375 complete. Loss:  0.2511582672595978
batch 376 complete. Loss:  0.25154244899749756
batch 377 complete. Loss:  0.2743097245693207
batch 378 complete. Loss:  0.2545631527900696
batch 379 complete. Loss:

batch 535 complete. Loss:  0.25649911165237427
batch 536 complete. Loss:  0.45399290323257446
batch 537 complete. Loss:  0.44427773356437683
batch 538 complete. Loss:  0.1768198013305664
batch 539 complete. Loss:  0.2860146164894104
batch 540 complete. Loss:  0.3898909091949463
batch 541 complete. Loss:  0.1550290286540985
batch 542 complete. Loss:  0.6847202181816101
batch 543 complete. Loss:  0.27459171414375305
batch 544 complete. Loss:  0.1618342101573944
batch 545 complete. Loss:  0.2286982536315918
batch 546 complete. Loss:  0.19392214715480804
batch 547 complete. Loss:  0.26226693391799927
batch 548 complete. Loss:  0.17934466898441315
batch 549 complete. Loss:  0.24101969599723816
batch 550 complete. Loss:  0.17916709184646606
batch 551 complete. Loss:  0.42498892545700073
batch 552 complete. Loss:  0.3544105291366577
batch 553 complete. Loss:  0.2525933086872101
batch 554 complete. Loss:  0.4075079560279846
batch 555 complete. Loss:  0.2367069125175476
batch 556 complete. Loss

batch 37 complete. Loss:  0.15473490953445435
batch 38 complete. Loss:  0.24518635869026184
batch 39 complete. Loss:  0.19926458597183228
batch 40 complete. Loss:  0.27507907152175903
batch 41 complete. Loss:  0.15115313231945038
batch 42 complete. Loss:  0.0951133519411087
batch 43 complete. Loss:  0.30076664686203003
batch 44 complete. Loss:  0.1771053969860077
batch 45 complete. Loss:  0.21281182765960693
batch 46 complete. Loss:  0.16515672206878662
batch 47 complete. Loss:  0.21139228343963623
batch 48 complete. Loss:  0.15574443340301514
batch 49 complete. Loss:  0.17259502410888672
batch 50 complete. Loss:  0.1846737414598465
batch 51 complete. Loss:  0.10747918486595154
batch 52 complete. Loss:  0.2539268434047699
batch 53 complete. Loss:  0.13314010202884674
batch 54 complete. Loss:  0.18235254287719727
batch 55 complete. Loss:  0.23017291724681854
batch 56 complete. Loss:  0.16651751101016998
batch 57 complete. Loss:  0.24774879217147827
batch 58 complete. Loss:  0.2142063081

...

batch 466 complete. Loss:  0.1593926101922989
batch 467 complete. Loss:  0.05039026215672493
batch 468 complete. Loss:  0.10189669579267502
batch 469 complete. Loss:  0.032705649733543396
batch 470 complete. Loss:  0.04228615015745163
batch 471 complete. Loss:  0.06562916934490204
batch 472 complete. Loss:  0.06542974710464478
batch 473 complete. Loss:  0.05222484469413757
batch 474 complete. Loss:  0.06676197052001953
batch 475 complete. Loss:  0.03927042335271835
batch 476 complete. Loss:  0.03660622984170914
batch 477 complete. Loss:  0.021692197769880295
batch 478 complete. Loss:  0.09231612831354141
batch 479 complete. Loss:  0.020512357354164124
batch 480 complete. Loss:  0.033023446798324585
batch 481 complete. Loss:  0.034610506147146225
batch 482 complete. Loss:  0.0804472267627716
batch 483 complete. Loss:  0.030571192502975464
batch 484 complete. Loss:  0.0827539712190628
batch 485 complete. Loss:  0.03997218608856201
batch 486 complete. Loss:  0.039648134261369705
batch 487

batch 640 complete. Loss:  0.03707797825336456
batch 641 complete. Loss:  0.043527133762836456
batch 642 complete. Loss:  0.030529530718922615
batch 643 complete. Loss:  0.04343430697917938
batch 644 complete. Loss:  0.05133861303329468
batch 645 complete. Loss:  0.05620146915316582
batch 646 complete. Loss:  0.03427629545331001
batch 647 complete. Loss:  0.05254673212766647
batch 648 complete. Loss:  0.04951692372560501
batch 649 complete. Loss:  0.030535470694303513
batch 650 complete. Loss:  0.063778817653656
batch 651 complete. Loss:  0.033567775040864944
batch 652 complete. Loss:  0.036160603165626526
batch 653 complete. Loss:  0.026018051430583
batch 654 complete. Loss:  0.03107631392776966
batch 655 complete. Loss:  0.07170046120882034
batch 656 complete. Loss:  0.027112405747175217
batch 657 complete. Loss:  0.039366960525512695
batch 658 complete. Loss:  0.01928875595331192
batch 659 complete. Loss:  0.06172054260969162
batch 660 complete. Loss:  0.09311246126890182
batch 661 

batch 140 complete. Loss:  0.025169258937239647
batch 141 complete. Loss:  0.027004579082131386
batch 142 complete. Loss:  0.02837042137980461
batch 143 complete. Loss:  0.09570339322090149
batch 144 complete. Loss:  0.027369268238544464
batch 145 complete. Loss:  0.046462737023830414
batch 146 complete. Loss:  0.026407141238451004
batch 147 complete. Loss:  0.03842812776565552
batch 148 complete. Loss:  0.09223732352256775
batch 149 complete. Loss:  0.08209720253944397
batch 150 complete. Loss:  0.023851847276091576
batch 151 complete. Loss:  0.0687013491988182
batch 152 complete. Loss:  0.023930057883262634
batch 153 complete. Loss:  0.030308710411190987
batch 154 complete. Loss:  0.03111257776618004
batch 155 complete. Loss:  0.034319665282964706
batch 156 complete. Loss:  0.021427158266305923
batch 157 complete. Loss:  0.03408297896385193
batch 158 complete. Loss:  0.04424801841378212
batch 159 complete. Loss:  0.025891687721014023
batch 160 complete. Loss:  0.05044376105070114
bat

batch 314 complete. Loss:  0.04251422733068466
batch 315 complete. Loss:  0.024670526385307312
batch 316 complete. Loss:  0.015822306275367737
batch 317 complete. Loss:  0.02470441535115242
batch 318 complete. Loss:  0.04569867253303528
batch 319 complete. Loss:  0.052855804562568665
batch 320 complete. Loss:  0.036889322102069855
batch 321 complete. Loss:  0.019825516268610954
batch 322 complete. Loss:  0.02646091766655445
batch 323 complete. Loss:  0.024306606501340866
batch 324 complete. Loss:  0.02704053558409214
batch 325 complete. Loss:  0.05146161466836929
batch 326 complete. Loss:  0.025921925902366638
batch 327 complete. Loss:  0.042284801602363586
batch 328 complete. Loss:  0.039526041597127914
batch 329 complete. Loss:  0.019645988941192627
batch 330 complete. Loss:  0.024212129414081573
batch 331 complete. Loss:  0.02151418849825859
batch 332 complete. Loss:  0.05137420445680618
batch 333 complete. Loss:  0.035945042967796326
batch 334 complete. Loss:  0.025608159601688385


KeyboardInterrupt: 

In [26]:
1

1

In [37]:
idx =101

#text=scaled_data['text'][idx]
text='''East Lansing Public Schools, home of Michigan State University (MSU) and a neighbor to the state capital, seeks a
dynamic individual to work in our highly diverse school district. This individual must love kids, have a passion for
their success, be a collaborative team player, deeply understand best practices for teaching and learning, and
engage all students in the learning process. Work in a district that has built six new elementary schools with stateof-the-art instructional and sustainable enhancements, has multiple partnerships with MSU, encourages
continuous professional growth and learning, and is highly supported by its families and community!

POSITION: Summer School Administrator (shared position)
LOCATION: Donley and Red Cedar Elementary Schools
SALARY: Stipend: $3,500
WORK YEAR: Weeks of June 24, July 8, July 15, Plus July 22 and July 25 - Work Days: M, T, W, TH 8:30 AM to
1:30 PM
START DATE: June 24, 2024
REPORTS TO: Assistant Superintendent
APPLICATION DEADLINE: March 15, 2024


GENERAL SUMMARY: Under the direct supervision of the Assistant Superintendent, and in accordance with the
established policies and procedures of the East Lansing Public School District, the Summer School Administrator is
responsible for the administration and oversight of the elementary summer school program. This person will be
sharing the administration with an elementary principal and working the weeks/days the elementary principal is not
working. The elementary summer school program will likely be run at two sites: Donley and Red Cedar, so travel
between sites will be necessary.'''
data = torch.tensor(nn_data[idx]).unsqueeze(0)

# inference
inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to('cuda')
outputs = model(**inputs, nn_data=data)
pred = scaler.inverse_transform(np.array([outputs.item()]))
print(pred)
#print(pred, scaler.inverse_transform(np.array([scaled_data['label'][idx]])))



[41296.7139462]


In [31]:
train_losses, val_losses

([0.4371773604441572,
  0.17498615956416835,
  0.1035798030457011,
  0.06832966648318149,
  0.05117192253332447],
 [0.2274242325647343,
  0.1909041110668662,
  0.11509220552294565,
  0.09375646160419523,
  0.08577173611027955])

In [53]:
train_losses = []
with torch.no_grad():  # Disable gradient calculation for validation
    for batch in train_dataloader:
        input_ids = batch['input_ids'].to('cuda') 
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')
        data = batch['nn_data'].to('cuda')
        for i in range(len(labels)):
            output = model(input_ids[i].unsqueeze(0), 
                           attention_mask=attention_mask[i].unsqueeze(0),
                           nn_data = data[i].unsqueeze(0))
            yhat = scaler.inverse_transform(np.array([output.squeeze(1).item()]))
            y=scaler.inverse_transform(np.array([labels[i].unsqueeze(0).item()]))
            loss = np.abs(yhat-y)
            
            train_losses.append(loss)
            #print(loss)


In [50]:
test_losses = []
with torch.no_grad():  # Disable gradient calculation for validation
    for batch in val_dataloader:
        input_ids = batch['input_ids'].to('cuda') 
        attention_mask = batch['attention_mask'].to('cuda')
        labels = batch['labels'].to('cuda')
        data = batch['nn_data'].to('cuda')
        for i in range(len(labels)):
            output = model(input_ids[i].unsqueeze(0), 
                           attention_mask=attention_mask[i].unsqueeze(0),
                           nn_data = data[i].unsqueeze(0))
            yhat = scaler.inverse_transform(np.array([output.squeeze(1).item()]))
            y=scaler.inverse_transform(np.array([labels[i].unsqueeze(0).item()]))
            loss = np.abs(yhat-y)
            
            test_losses.append(loss)
            
        


[9902.94945394]
[8213.52738406]
[12467.39796687]
[260.93463656]
[17561.46801619]
[3606.91381816]
[4953.99896561]
[3721.92276343]
[21343.3596639]
[7007.92769165]
[7188.05140909]
[2524.72952969]
[9134.87295422]
[6684.25142006]
[1175.89789443]
[4394.2467458]
[6955.62663285]
[5130.97102807]
[6395.97431644]
[5420.8067014]
[1216.85330206]
[11616.99569804]
[5326.50096166]
[9762.26706114]
[19189.5766286]
[1046.47831543]
[18312.76151205]
[1703.36942797]
[60.52036639]
[226.75962078]
[1411.82485044]
[19815.49960526]
[1227.31688867]
[3260.25171138]
[9009.20176606]
[2303.59057763]
[7811.01678496]
[1779.35276944]
[2559.29188686]
[11768.05116995]
[6114.27818138]
[18828.49775199]
[4871.48144334]
[505.41531768]
[7973.72241207]
[11857.54543931]
[4596.51661133]
[4307.18103025]
[6905.18481273]
[8969.07780014]
[16275.40179334]
[9364.40992782]
[33615.88195896]
[1752.80109243]
[2891.96291583]
[2858.09087005]
[6281.40717046]
[1138.7192571]
[20420.86664281]
[539.67930693]
[21385.78543479]
[12986.11942005]
[776

[1041.33051001]
[2530.73562227]
[11783.45431806]
[8815.05552328]
[15824.14600611]
[7958.48954078]
[2277.15471953]
[390.08882918]
[6251.27929695]
[536.63273267]
[1425.81669526]
[4073.20228638]
[2242.13752386]
[6958.74990838]
[3221.92715715]
[5640.87183574]
[17557.6620994]
[5106.45998821]
[687.46423689]
[6305.82271724]
[2423.0850129]
[7890.03672953]
[44283.27764174]
[5635.28875064]
[6548.60676637]
[53836.55062917]
[11472.35551995]
[24777.60436059]
[3785.00802012]
[5044.93675278]
[7263.24204299]
[13100.12204472]
[2924.88309891]
[16394.17523671]
[3477.52261863]
[24170.89121583]
[7266.04739177]
[5136.05171988]
[12799.02355759]
[1881.52364867]
[6243.10562661]
[6965.83940633]
[7272.87533841]
[460.90710744]
[3087.32103859]
[11812.78641594]
[564.21450767]
[6960.78249191]
[48368.97456423]
[19796.63876412]
[8688.23765121]
[1307.19205235]
[4685.11270888]
[4649.02668108]
[9923.57749223]
[9947.79668377]
[48286.31821268]
[9647.88016314]
[1816.7216259]
[2096.57999864]
[599.11051534]
[8306.46247218]
[5

[3290.16520485]
[18735.26813101]
[1159.59427347]
[2774.52407448]
[14946.2601399]
[967.05568922]
[3542.96950729]
[12452.45042428]
[1869.94616793]
[5126.26003635]
[10093.29745005]
[2514.93017604]
[12469.24186528]
[3876.7826181]
[381.42005217]
[7430.48416643]
[1808.4348212]
[3516.097986]
[1018.0160099]
[5741.574461]
[27616.89421649]
[4054.56119436]
[8078.72337624]
[9238.94660009]
[9179.63197759]
[5858.9979621]
[5865.89800793]
[1285.64820095]
[6440.51013913]
[68316.04598599]
[3.46076109]
[8217.32639773]
[4719.84304181]
[9450.8507198]
[5110.42736116]
[6270.12288031]
[15621.7896599]
[3372.99864046]
[12809.42424917]
[1780.28238878]
[3103.348535]
[7365.61138675]
[2102.68235131]
[1654.74389381]
[8170.76872968]
[4280.92465311]
[2620.96200521]
[30536.77370835]
[1952.67768673]
[9106.97440298]
[363.82478191]
[27395.3111641]
[7091.39400857]
[10132.91825566]
[3134.03824527]
[9543.35244578]
[13793.14252127]
[3190.45355994]
[44057.72223082]
[1488.62045832]
[5874.84137547]
[7973.74695647]
[3961.87193379

[830.32494562]
[2436.46631564]
[229.41348456]
[1161.02398506]
[4005.98129745]
[6782.86700389]
[13136.48994967]
[1401.44870327]
[3267.82059224]
[17221.36693564]
[1830.33149842]
[4992.32582088]
[6750.74221294]
[4226.67001309]
[1775.42873271]
[9740.01679109]
[6916.74829549]
[14920.25227486]
[9564.69994194]
[22924.87762672]
[3715.06567031]
[10442.37909825]
[4266.71651072]
[3977.62944174]
[1891.07890055]
[7111.01879431]
[1063.3280494]
[7890.53835581]
[4938.93560412]
[29097.41041911]
[3256.97196527]
[6231.80753023]
[9103.71153116]
[16425.72323415]
[6107.44870072]
[24176.79107716]
[14599.23370211]
[8322.92793281]
[1664.55245161]
[3511.76436451]
[5598.03571309]
[16021.3725717]
[20274.06732411]
[3.34436692]
[12444.6380169]
[17170.63978681]
[7141.51981948]
[112516.26459453]
[6163.11234299]
[28036.2698898]
[10825.62118897]
[34287.93230957]
[17102.48304245]
[9532.93641395]
[2842.03346016]
[9401.60658994]
[1867.91205038]
[12275.08488433]
[378.11882971]
[3799.60273689]
[8941.40091557]
[947.28517107]

[11741.24791279]
[7436.17156526]
[1843.69285884]
[12187.69913276]
[2131.06488753]
[13043.84172428]
[3393.47260508]
[11191.71632833]
[11132.76526984]
[5017.00675403]
[5154.34650568]
[1938.57232404]
[3618.80251428]
[2896.12626051]
[5489.26104667]
[6430.42775781]
[3114.56532804]
[7392.39125003]
[1310.45339015]
[3411.0648073]
[4586.23557373]
[235.84411865]
[2660.80984657]
[35860.54142042]
[1321.10872993]
[2957.38295905]
[9609.74812909]
[14548.76043447]
[7617.74263558]
[9989.27212598]
[1370.68535978]
[6161.173335]
[3984.07541608]
[6444.92199591]
[13134.5578448]
[3234.5567875]
[5183.99307875]
[2149.43484057]
[1915.04957999]
[10155.02662838]
[6000.7955913]
[75283.86275094]
[520.2800229]
[13312.65818287]
[10020.38522724]
[8158.61541419]
[8390.16713828]
[877.07590085]
[3594.98983945]
[879.45977618]
[4637.11612509]
[333.41273021]
[3337.7107852]
[20723.76147358]
[3261.22658446]
[22517.90224697]
[9948.80147035]
[4524.47264685]
[2901.84433984]
[21553.13992558]
[360.91626993]
[13593.2436836]
[30985.

...

[11668.16464616]
[5568.02711005]
[2751.68703974]
[1238.16091356]
[6031.92249879]
[3475.56443533]
[8154.67910526]
[5114.3759423]
[4563.48291039]
[4063.95978391]
[2298.28591812]
[126.52947532]
[307.31128962]
[9116.98545213]
[4804.50819951]
[951.8933831]
[3204.70158699]
[14758.82161453]
[24060.16833727]
[9226.88609313]
[8371.01655811]
[9932.88749182]
[31669.04430657]
[2835.29908907]
[3973.6808606]
[11402.5082797]
[52.86864816]
[2129.37439164]
[456.6670615]
[2656.27450075]
[8625.06342104]
[6719.92057648]
[11602.63108507]
[1060.32442785]
[197.15676744]
[8632.23959143]
[1890.89865258]
[10928.21219939]
[6629.60023449]
[2088.67976831]
[3873.86490197]
[6617.28507934]
[7445.05740684]
[1855.22259304]
[1314.04607742]
[2200.57157416]
[5325.04056957]
[11548.72236775]
[397.36164315]
[6093.38782476]
[57652.10109314]
[992.55732592]
[5262.55971884]
[4248.33275145]
[17421.59712278]
[4076.45135198]
[7355.1880683]
[15490.06674191]
[17049.1664589]
[4407.81309035]
[1688.75323485]
[6446.71987357]
[11216.40032

[15338.13073947]
[8749.12733419]
[6004.64139274]
[10315.97963157]
[13113.12444322]
[959.63100674]
[388.06238175]
[2963.76143627]
[6025.43050369]
[3733.29142497]
[347.83410211]
[10653.01419553]
[12839.22115672]
[14018.19016981]
[8920.96693147]
[354.08985731]
[1057.0845664]
[5541.66378258]
[1323.4956733]
[5287.19002916]
[2448.40256655]
[19404.45215533]
[7714.06715262]
[420.78927763]
[76007.87974283]
[2736.15810157]
[12800.19248487]
[9971.82719019]
[30019.22923608]
[5433.62808491]
[7582.3005149]
[9243.07926426]
[1462.171561]
[13276.78845263]
[951.37795059]
[12333.78129463]
[23056.26536018]
[1715.84258773]
[17342.30718976]
[20889.18462651]
[12862.4002791]
[5024.86863373]
[6856.23713325]
[1265.17970128]
[16225.56054413]
[18528.94863042]
[4766.27875485]
[16581.89621449]
[15941.15225233]
[9690.9471566]
[1403.38157516]
[4494.36586618]
[576.8548762]
[22016.85046427]
[8876.83532445]
[7803.79536083]
[12275.95697772]
[6878.83332602]
[3854.41652913]
[10678.60173764]
[8695.2266705]
[3417.96216858]
[

[577.873469]
[11041.41971956]
[40328.31035585]
[7536.52826781]
[15968.03757985]
[2595.41434776]
[5990.63267364]
[8370.84167922]
[21915.11160492]
[4628.2118752]
[414.66544861]
[10140.75405693]
[3538.47481314]
[3313.78919457]
[33978.30464174]
[6678.74426921]
[22104.9180901]
[76163.74285006]
[10770.46837713]
[18743.23662546]
[10085.82828084]
[14841.13645371]
[5653.02438422]
[5351.13932561]
[4029.71803803]
[1574.06950294]
[5288.13537226]
[7511.34417436]
[13693.15820336]
[11155.54554564]
[5978.92762911]
[839.03053923]
[66328.93173217]
[27280.16569058]
[1368.76782815]
[2655.28428742]
[53.66634132]
[5140.77651782]
[11143.6016246]
[1143.11884168]
[2543.88643772]
[6658.29877993]
[10386.47883258]
[5277.42442406]
[1309.51916874]
[13301.23276238]
[16956.81199936]
[786.77089912]
[2075.99031098]
[1387.80815025]
[2849.70665474]
[7386.57115801]
[6299.84308659]
[34719.4935129]
[17156.03279785]
[1958.6527153]
[15902.20219928]
[47844.4368541]
[12214.40344529]
[3821.52702657]
[21156.31135622]
[14106.71877

[27414.74112863]
[315.60423043]
[1822.08764642]
[9353.05353847]
[3037.29033725]
[1278.98286099]
[202.71224008]
[12696.12727614]
[4658.31328677]
[35740.64468702]
[12789.25488444]
[3140.25028075]
[9074.47454284]
[5551.85081319]
[5861.12258715]
[276.25341327]
[2538.75972515]
[1537.7867364]
[3476.06452758]
[17270.489494]
[3088.83320404]
[7444.11014622]
[2544.35891751]
[1530.62590627]
[1239.00223057]
[11416.771647]
[7712.48097045]
[29070.17610084]
[1081.4832389]
[137.30216803]
[17998.09610511]
[4590.07984115]
[15227.6740143]
[2493.49677446]
[4135.86645313]
[2205.08467661]
[8192.83031463]
[19079.41213525]
[5197.52011391]
[4829.30571858]
[4968.73864777]
[26677.33209582]
[11176.59850894]
[3464.90909555]
[3602.49275723]
[4431.19853913]
[11883.9743943]
[756.21618307]
[5014.4526019]
[7511.4316138]
[18306.09190558]
[4366.04311528]
[7521.98455739]
[28449.82470409]
[10526.73016426]
[4402.14946892]
[22339.24352369]
[11170.36729815]
[50263.29639428]
[12994.541986]
[4269.56366169]
[3254.18310727]
[4059

[2312.74257261]
[9501.86012929]
[1948.62325784]
[5187.40935311]
[701.96998018]
[9886.27613288]
[2216.04682145]
[6177.4796405]
[4048.86190684]
[2669.14267203]
[5636.06113238]
[632.32599829]
[7189.58696843]
[1986.16929423]
[63.02389569]
[5577.67919727]
[29974.98948027]
[2942.42007621]
[10697.32911858]
[1475.46849235]
[2956.62515054]
[3083.44302262]
[1029.00193213]
[11153.45313512]
[3307.24811066]
[4007.91455285]
[3377.63225977]
[2451.10245109]
[3158.23596046]
[2844.52625128]
[5975.86307799]
[15827.776277]
[837.00102375]
[6775.47990503]
[1184.66945113]
[21694.4117676]
[13950.13045255]
[2427.07002713]
[7575.3659535]
[2383.40284627]
[1131.96954576]
[995.26334656]
[6022.48210705]
[14087.95150464]
[2681.80758496]
[714.70852633]
[28523.5008715]
[13125.85225118]
[6549.89688165]
[1895.84204913]
[1840.88252448]
[3724.75150609]
[42466.23693857]
[7394.48672859]
[4408.37224257]
[12459.69102373]
[347.58712404]
[9042.82146467]
[2257.79455315]
[75.01383748]
[936.98572517]
[10393.30677921]
[3813.4181688

In [54]:
print("Mean training err: " +  str(np.mean(train_losses))
print("Median training err:",np.median(train_losses))
print("Mean test err:",np.mean(test_losses))
print("Median test err:",np.median(test_losses))

Mean training err: 7439.551102733164
Median training err: 5317.480125849277
Mean test err: 9738.96757151688
Median test err: 6294.486270221943


In [ ]:
#del model
#torch.cuda.empty_cache()